In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import math
from PIL import Image
torch.manual_seed(0)

In [2]:
class PoseDataset(Dataset):    
    def __init__(self, data_dir = None):
        self._data_dir = data_dir
        data_dirs = [x[0] for x in os.walk(self._data_dir)][1:]
        annotations = []
        for i in range(len(data_dirs)):
            data = json.load(open(f"{data_dirs[i]}/annotations.json"))
            annotation = data['annotations']
            img_dict = {}
            for name in data['images']:
                img_dict[name['id']] = name['file_name']
            for k in range(len(annotation)):
                annotation[k]['image_path'] = f"{data_dirs[i]}/{img_dict[annotation[k]['image_id']]}"
            annotations.extend(annotation)
        self._annotations_person = [ano for ano in annotations if ano['category_id'] == 0 
                                    and len(ano['keypoints'])==51 and ano['pose_category']!="lifting"]
        self._pose_dict = {"reachingbad": 0, 
                           "reachinggood": 1, 
                           'randomrandom': 1}

    def _process_pose(self, pose):
        return self._pose_dict[pose]
    
    def _get_pose_size(self, keypoints, ratio):
        hips_center = (keypoints[9,:] + keypoints[10,:]) / 2
        shoulders_center = (keypoints[3,:] + keypoints[4,:]) / 2
        torso_size = np.linalg.norm((shoulders_center - hips_center))
        distance = np.linalg.norm((keypoints - hips_center), axis = 1)
        max_d = np.max(distance)
        pose_size = max(torso_size * ratio, max_d)
        return pose_size
    def _normalize_pose(self,keypoints):
        data_p = np.expand_dims(np.array(keypoints), axis=1).reshape(-1,3)[:,0:2]
        data_p = np.delete(data_p,[3,4], axis = 0)
        hip_center = (data_p[9,:] + data_p[10,:]) / 2
        data_p = data_p - hip_center
        pose_size = self._get_pose_size(data_p, 2)
        data_p = data_p / pose_size
        return data_p.flatten()
    def __getitem__(self, idx):
        actor = self._annotations_person[idx]
        if actor['keypoints'] is None:
            print("is None")
        x = torch.tensor(self._normalize_pose(actor['keypoints']))
        pose = actor['pose_category'] + actor['pose_subcategory']
        y = self._process_pose(pose)
        return x, y, actor['image_path'], np.array(actor['bbox'])
    def __len__(self):
        return len(self._annotations_person)

In [3]:
pose_data = PoseDataset(data_dir = '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/')

In [4]:
print(len(pose_data))

39891


In [5]:
num_val= int(len(pose_data) * 0.1)
num_train = len(pose_data) - 2 * num_val
train, val, test = random_split(pose_data, [num_train, num_val, num_val])
batch_size = 128
dataloaders = {
    'train': DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8),
    'val': DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8),
}
dataset_sizes = {}
dataset_sizes['train'] = len(train)
dataset_sizes['val'] = len(val)
dataset_sizes['test'] = len(test)
print(dataset_sizes)

{'train': 31913, 'val': 3989, 'test': 3989}


In [6]:
print(f"number of datapoints: {len(pose_data)}")

number of datapoints: 39891


In [7]:
from torch import nn
class PoseModel(nn.Module):
    
    def __init__(self, input_size, num_classes):
        super(PoseModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        y = x.clone().detach()
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.softmax(x)
        return x, y

In [8]:
from torch import nn
import wandb
from torch.optim import lr_scheduler
import time
import copy
from datetime import datetime


def train_model(model, dataloaders, dataset_sizes, optimizer, config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = nn.CrossEntropyLoss()
    wandb.init(project =  "ergo_ml_training", entity = "voxel-wandb", config = config, tags = [config['tags']])
    scheduler = lr_scheduler.StepLR(optimizer, step_size=config['step'], gamma=0.1)
    config = wandb.config
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())   
    best_accuracy = 0
    model.to(device)
    today_date = datetime.today().strftime('%Y-%m-%d')
    for epoch in range(config.num_epochs):
        print('Epoch {}/{}'.format(epoch, config.num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels, _, _ in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs, y = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase] 
            if phase == 'train':
                scheduler.step()
                
            if phase == 'val':
                #print("val accuracy", epoch_acc)
                wandb.log({"val loss":epoch_loss})
                wandb.log({"val_accuracy":epoch_acc})
            if phase == 'train':
                #print("train accuracy", epoch_acc)
                #print("train loss", epoch_loss)
                wandb.log({"train loss":epoch_loss})
                wandb.log({"train_accuracy":epoch_acc})
            print(f'{phase} Loss: {epoch_loss} Acc: {epoch_acc}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_accuracy:
                best_accuracy = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_accuracy))

    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), f"voxel_ergo_ml_{config.tags}_{today_date}.pth")
    wandb.join()
    return model
    

In [9]:
config = {
        'num_epochs': 100,
        'tags': "ergoMLLinearAlphaPoseModelOverreaching",
        'step': 70,
}
pose_model = PoseModel(30,2)
optimizer = torch.optim.SGD(pose_model.parameters(), lr=0.1, momentum=0.9)
train_model(pose_model, dataloaders, dataset_sizes, optimizer, config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: voxel-wandb (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-05-23 17:10:11.135427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 17:10:11.135477: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 0/99
----------
train Loss: 0.43309016281680685 Acc: 0.8807695923291449
val Loss: 0.34760062086848154 Acc: 0.9639007269992479
Epoch 1/99
----------
train Loss: 0.3479042258905654 Acc: 0.9657819697302039
val Loss: 0.3457638880503629 Acc: 0.9656555527701177
Epoch 2/99
----------
train Loss: 0.34484612985353275 Acc: 0.9685394666750227
val Loss: 0.3455205225920611 Acc: 0.9661569315617948
Epoch 3/99
----------
train Loss: 0.3438969982522808 Acc: 0.9690408297558988
val Loss: 0.3428679918041502 Acc: 0.9699172724993732
Epoch 4/99
----------
train Loss: 0.3434600686814431 Acc: 0.9694168520665559
val Loss: 0.3439339399188346 Acc: 0.9684131361243419
Epoch 5/99
----------
train Loss: 0.3420197989255712 Acc: 0.9708896061166296
val Loss: 0.3430532626443472 Acc: 0.9699172724993732
Epoch 6/99
----------
train Loss: 0.3408994084556851 Acc: 0.971986337856046
val Loss: 0.3400177029656061 Acc: 0.9724241664577588
Epoch 7/99
----------
train Loss: 0.34017899706509136 Acc: 0.9726757120922508
val Loss: 

val Loss: 0.32640970815659526 Acc: 0.986964151416395
Epoch 63/99
----------
train Loss: 0.3271820122241316 Acc: 0.9860245041205777
val Loss: 0.33067185530898024 Acc: 0.9827024316871396
Epoch 64/99
----------
train Loss: 0.32713783413850683 Acc: 0.9859304985429135
val Loss: 0.3267762278383852 Acc: 0.9864627726247179
Epoch 65/99
----------
train Loss: 0.32837394430588573 Acc: 0.9845517500705041
val Loss: 0.32645809761740924 Acc: 0.9859613938330408
Epoch 66/99
----------
train Loss: 0.3283881614788934 Acc: 0.9845517500705041
val Loss: 0.32873646682980484 Acc: 0.9839558786663324
Epoch 67/99
----------
train Loss: 0.3291198883573477 Acc: 0.9835803591013067
val Loss: 0.32727356393643864 Acc: 0.9854600150413637
Epoch 68/99
----------
train Loss: 0.328554274070208 Acc: 0.9844264093002851
val Loss: 0.32578630996067204 Acc: 0.9874655302080722
Epoch 69/99
----------
train Loss: 0.32745360728679296 Acc: 0.9855858114248112
val Loss: 0.3249925949066078 Acc: 0.9879669089997493
Epoch 70/99
----------


train loss,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇███████████████████
val loss,█▇▇▆▅▅▄▄▃▄▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▃▃▄▅▅▅▆▆▆▆▆▆▇▇▆▇▆▆▇▇▇▇▇▆▆▇████████████
train loss,0.32205
train_accuracy,0.99138
val loss,0.32414
val_accuracy,0.98872


PoseModel(
  (layer1): Linear(in_features=30, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)

In [10]:
pose_data_real = PoseDataset(data_dir = '/home/reza_voxelsafety_com/experiments/ergonomic/realdata/')
print(len(pose_data_real))

5227


In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
wandb.init(
            project="ergo_synthetic_test",
            job_type="inference",
            entity="voxel-wandb",
        )
TEST_TABLE_NAME = "test_results"
columns = ["Images", "Prediction", "GT"]
labels_type= ["reachingbad", "reachinggoodrandomrandom"]
for klass in labels_type:
    columns.append("score_" + klass)
test_dt = wandb.Table(columns=columns)
model = PoseModel(30, 2)
model.load_state_dict(
    torch.load('/home/reza_voxelsafety_com/voxel/experimental/reza/Ergonomic/voxel_ergo_ml_ergoMLLinearAlphaPoseModelOverreaching_2022-05-23.pth')
)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataloaders_test = DataLoader(pose_data_real, batch_size=1, shuffle=False, num_workers=8)
running_corrects = 0
model.to(device)
y_true = []
y_pred = []
for inputs, labels, path, bbox in dataloaders_test:
    bbox = bbox.tolist()[0]
    image = Image.open(path[0])
    image = image.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs,_ = model(inputs.float())
    _, preds = torch.max(outputs, 1)
    y_pred.append(preds.item())
    y_true.append(labels.item())
    row = [wandb.Image(image),
            labels_type[preds], labels_type[labels]]
    for c_i in outputs.data.cpu().numpy().tolist():
        for c in c_i:
            row.append(np.round(c, 4))
    test_dt.add_data(*row)
    running_corrects += torch.sum(preds == labels.data)

label_mapping = {
   "reachingbad": 0, 
   "reachinggood": 1, 
   'randomrandom': 1
}
target_names = list(label_mapping.keys())

cm = confusion_matrix(y_pred,y_true)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
# disp.plot(xticks_rotation=45)
wandb.sklearn.plot_confusion_matrix(y_true, y_pred, target_names)
wandb.log({TEST_TABLE_NAME: test_dt})
accuracy_test = running_corrects.double() / len(pose_data_real)
print(f"Accuracy test {accuracy_test}")

wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-05-23 17:23:40.553077: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 17:23:40.553129: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Accuracy test 0.98890376889229


In [13]:
accuracy_test = running_corrects.double() / len(test)
print(f"Accuracy test {accuracy_test}")

Accuracy test 0.9899724241664577


In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

  liftingbad       0.16      0.45      0.24       228
 liftinggood       0.00      0.00      0.00         0
 reachingbad       0.14      0.38      0.21        40
reachinggood       0.17      0.04      0.07      1189
randomrandom       0.79      0.79      0.79      3998

    accuracy                           0.61      5455
   macro avg       0.25      0.33      0.26      5455
weighted avg       0.62      0.61      0.60      5455



/home/reza_voxelsafety_com/.cache/bazel/_bazel_reza_voxelsafety_com/d620f0b6d4b41da06538aa48e57dfab6/execroot/voxel/bazel-out/k8-fastbuild/bin/third_party/jupyter/jupyter.runfiles/pip_deps_scikit_learn/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/reza_voxelsafety_com/.cache/bazel/_bazel_reza_voxelsafety_com/d620f0b6d4b41da06538aa48e57dfab6/execroot/voxel/bazel-out/k8-fastbuild/bin/third_party/jupyter/jupyter.runfiles/pip_deps_scikit_learn/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/reza_voxelsafety_com/.cache/bazel/_bazel_reza_voxelsafety_com/

In [14]:
import functools
import os.path
import random
import sys
import xml.etree.ElementTree
import numpy as np
import matplotlib.pyplot as plt
import skimage.data
import cv2
import PIL.Image

wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
